In [1]:
import requests
from requests.exceptions import HTTPError
import os
from pyzbar.pyzbar import decode
from PIL import Image
import isbnlib
import streamlit as st
import time
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
GOOGLE_BOOKS_API_KEY = os.getenv("GOOGLE_BOOKS_API_KEY")

In [3]:
def get_google_book_by_isbn(isbn: str) -> dict | None:
    try:
        service = build("books", "v1", developerKey=GOOGLE_BOOKS_API_KEY)
        request = service.volumes().list(q=f"isbn:{isbn}")
        res = request.execute()
        if "items" in res:
            return res["items"][0]["volumeInfo"]
        else:
            return {}
    except HttpError as e:
        return {"error": f"An error occurred while fetching the book information.\n{e}"}

In [12]:
def get_basic_info(isbn: str):
    """Get a Book's Basic Information.

    Retrieves book information based on the provided ISBN.

    Parameters:
        isbn (str): The ISBN of the book.
    Returns:
        dict: A dictionary containing the book information.

    """
    book_info = {}
    book_info_unclean = get_google_book_by_isbn(isbn)
    book_info = {
        "Title": book_info_unclean.get("title", ""),
        "Authors": book_info_unclean.get("authors", []),
        "Publisher": book_info_unclean.get("publisher", ""),
        "Year": book_info_unclean.get("publishedDate", "").split("-")[0],
        "description": book_info_unclean.get("description", ""),
        "pageCount": book_info_unclean.get("pageCount", ""),
        "categories": book_info_unclean.get("categories", []),
        "averageRating": book_info_unclean.get("averageRating", ""),
        "thumbnail": book_info_unclean.get("thumbnail", ""),
        "infoLink": book_info_unclean.get("infoLink", ""),
    }
    return book_info

In [4]:
isbn_tests = ["9781982110574", "9781668022283", "9788433966223"]

In [5]:
for isbn in isbn_tests:
    print(get_google_book_by_isbn(isbn))

{'title': 'The Institute', 'subtitle': 'A Novel', 'authors': ['Stephen King'], 'publisher': 'Simon and Schuster', 'publishedDate': '2021-06-29', 'description': 'In the middle of the night, in a house on a quiet street in suburban Minneapolis, intruders silently murder Luke Ellis\' parents and load him into a black SUV. The operation takes less than two minutes. Luke will wake up at The Institute, in a room that looks just like his own, except there\'s no window. And outside his door are other doors, behind which are other kids with special talents--telekinesis and telepathy--who got to this place the same way Luke did: Kalisha, Nick, George, Iris, and 10-year-old Avery Dixon. They are all in Front Half. Others, Luke learns, graduated to Back Half, "like the roach motel," Kalisha says. "You check in, but you don\'t check out." In this most sinister of institutions, the director, Mrs. Sigsby, and her staff are ruthlessly dedicated to extracting from these children the force of their extr

In [13]:
for isbn in isbn_tests:
    print(get_basic_info(isbn))

{'Title': 'The Institute', 'Authors': ['Stephen King'], 'Publisher': 'Simon and Schuster', 'Year': '2021', 'description': 'In the middle of the night, in a house on a quiet street in suburban Minneapolis, intruders silently murder Luke Ellis\' parents and load him into a black SUV. The operation takes less than two minutes. Luke will wake up at The Institute, in a room that looks just like his own, except there\'s no window. And outside his door are other doors, behind which are other kids with special talents--telekinesis and telepathy--who got to this place the same way Luke did: Kalisha, Nick, George, Iris, and 10-year-old Avery Dixon. They are all in Front Half. Others, Luke learns, graduated to Back Half, "like the roach motel," Kalisha says. "You check in, but you don\'t check out." In this most sinister of institutions, the director, Mrs. Sigsby, and her staff are ruthlessly dedicated to extracting from these children the force of their extranormal gifts. There are no scruples h

In [11]:
book_dict = get_google_book_by_isbn("9788433966223")
book_dict.keys()

dict_keys(['title', 'subtitle', 'authors', 'publisher', 'publishedDate', 'description', 'industryIdentifiers', 'readingModes', 'pageCount', 'printType', 'categories', 'maturityRating', 'allowAnonLogging', 'contentVersion', 'panelizationSummary', 'imageLinks', 'language', 'previewLink', 'infoLink', 'canonicalVolumeLink'])

In [7]:
for key, value in book_dict.items():
    print(f"{key}: {value}")

title: Novecento
subtitle: La leyenda del pianista en el océano
authors: ['Alessandro Baricco']
publisher: Anagrama
publishedDate: 1999-02
description: En los años de entreguerras, un transatlántico, el Virginian, recorría las rutas entre Europa y América, con su carga de millonarios, de turistas, de emigrantes... en el Virginian tocaba cada noche un pianista extraordinario, llamado Novecento, con una té
industryIdentifiers: [{'type': 'ISBN_10', 'identifier': '8433966227'}, {'type': 'ISBN_13', 'identifier': '9788433966223'}]
readingModes: {'text': False, 'image': False}
pageCount: 0
printType: BOOK
categories: ['Fiction']
maturityRating: NOT_MATURE
allowAnonLogging: False
contentVersion: preview-1.0.0
panelizationSummary: {'containsEpubBubbles': False, 'containsImageBubbles': False}
imageLinks: {'smallThumbnail': 'http://books.google.com/books/content?id=1ExPSAAACAAJ&printsec=frontcover&img=1&zoom=5&source=gbs_api', 'thumbnail': 'http://books.google.com/books/content?id=1ExPSAAACAAJ&pr